In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("automation-protocol.csv")

In [3]:
df.columns = df.columns.str.lower()
df.head()

,id,instanceid,type,lastseen,onoff,color,dimmer,protocoltime
0,270,65537,2,1605867202,1,efd275,51,2020-11-20T21:58:04.000Z
1,271,65538,2,1605868423,1,efd275,48,2020-11-20T21:58:04.000Z
2,272,65540,2,1605882795,0,efd275,100,2020-11-20T21:58:04.000Z
3,273,65545,2,1605831076,0,efd275,50,2020-11-20T21:58:04.000Z
4,274,65546,2,1605892658,0,f1e0b5,50,2020-11-20T21:58:04.000Z


## Data cleaning

In [4]:
df.protocoltime = pd.to_datetime(df.protocoltime)
df.head()

,id,instanceid,type,lastseen,onoff,color,dimmer,protocoltime
0,270,65537,2,1605867202,1,efd275,51,2020-11-20 21:58:04+00:00
1,271,65538,2,1605868423,1,efd275,48,2020-11-20 21:58:04+00:00
2,272,65540,2,1605882795,0,efd275,100,2020-11-20 21:58:04+00:00
3,273,65545,2,1605831076,0,efd275,50,2020-11-20 21:58:04+00:00
4,274,65546,2,1605892658,0,f1e0b5,50,2020-11-20 21:58:04+00:00


In [5]:
df.protocoltime = df.protocoltime.round('15min')
df.head()

,id,instanceid,type,lastseen,onoff,color,dimmer,protocoltime
0,270,65537,2,1605867202,1,efd275,51,2020-11-20 22:00:00+00:00
1,271,65538,2,1605868423,1,efd275,48,2020-11-20 22:00:00+00:00
2,272,65540,2,1605882795,0,efd275,100,2020-11-20 22:00:00+00:00
3,273,65545,2,1605831076,0,efd275,50,2020-11-20 22:00:00+00:00
4,274,65546,2,1605892658,0,f1e0b5,50,2020-11-20 22:00:00+00:00


### Data enriching

In [6]:
devices = df.instanceid.unique()
devices

array([65537, 65538, 65540, 65545, 65546, 65547, 65548, 65549, 65551,
       65552, 65553, 65554, 65555, 65556, 65557])

In [156]:
absolute_min_date = df.protocoltime.min(axis=0)
absolute_max_date = df.protocoltime.max(axis=0)
print("absolute_min_date=%s, absolute_max_date=%s" % (absolute_min_date, absolute_max_date))

complete_time_data = pd.date_range(absolute_min_date, absolute_max_date, freq="15min")
complete_time_data = complete_time_data.tz_localize(None)

complete_arr = []

for device in devices:
    single_device = df[df["instanceid"] == device]
    single_device["protocoltime"] = pd.to_datetime(single_device["protocoltime"]).apply(lambda x: x.replace(tzinfo=None))
    last_state = single_device.iloc[0]
    last_state["onoff"] = 0
    for time_data in complete_time_data.values:
        if time_data in single_device.protocoltime.values:
            # TODO edge case for multiple events in one time slot
            device_row = single_device[single_device["protocoltime"] == time_data]
            complete_arr.append(device_row.values[0])
            last_state = device_row.iloc[0]
        else:
            current_state = last_state.copy()
            current_state["protocoltime"] = time_data
            complete_arr.append(current_state.values)

complete_df = pd.DataFrame(complete_arr, columns=df.columns)
complete_df = complete_df.sort_values(by=["instanceid", "protocoltime"])
complete_df.to_csv('test.csv', index=False)
complete_df.head(60)

absolute_min_date=2020-11-20 22:00:00+00:00, absolute_max_date=2020-11-21 20:30:00+00:00
True


,id,instanceid,type,lastseen,onoff,color,dimmer,protocoltime
0,270,65537,2,1605867202,1,efd275,51,2020-11-20 22:00:00
1,270,65537,2,1605867202,1,efd275,51,2020-11-20 22:15:00
2,285,65537,2,1605867202,1,efd275,51,2020-11-20 22:30:00
3,285,65537,2,1605867202,1,efd275,51,2020-11-20 22:45:00
4,304,65537,2,1605867202,0,f1e0b5,51,2020-11-20 23:00:00
5,304,65537,2,1605867202,0,f1e0b5,51,2020-11-20 23:15:00
6,304,65537,2,1605867202,0,f1e0b5,51,2020-11-20 23:30:00
7,304,65537,2,1605867202,0,f1e0b5,51,2020-11-20 23:45:00
8,304,65537,2,1605867202,0,f1e0b5,51,2020-11-21 00:00:00
9,304,65537,2,1605867202,0,f1e0b5,51,2020-11-21 00:15:00


In [70]:
df["weekday"] = df.protocoltime.dt.day_name()
df["weekofyear"] = df.protocoltime.dt.weekofyear
df["year"] = df.protocoltime.dt.year
df["month"] = df.protocoltime.dt.month
df["day"] = df.protocoltime.dt.day
df["time"] = df.protocoltime.dt.strftime("%H:%M")
df.head()

,id,instanceid,type,lastseen,onoff,color,dimmer,protocoltime,weekday,weekofyear,year,month,day,time
0,270,65537,2,1605867202,1,efd275,51,2020-11-20 22:00:00+00:00,Friday,47,2020,11,20,22:00
1,271,65538,2,1605868423,1,efd275,48,2020-11-20 22:00:00+00:00,Friday,47,2020,11,20,22:00
2,272,65540,2,1605882795,0,efd275,100,2020-11-20 22:00:00+00:00,Friday,47,2020,11,20,22:00
3,273,65545,2,1605831076,0,efd275,50,2020-11-20 22:00:00+00:00,Friday,47,2020,11,20,22:00
4,274,65546,2,1605892658,0,f1e0b5,50,2020-11-20 22:00:00+00:00,Friday,47,2020,11,20,22:00


In [71]:
df.columns

Index(['id', 'instanceid', 'type', 'lastseen', 'onoff', 'color', 'dimmer',
       'protocoltime', 'weekday', 'weekofyear', 'year', 'month', 'day',
       'time'],
      dtype='object')

In [72]:
if "id" in df.columns:
    del df["id"]
if "lastseen" in df.columns:
    del df["lastseen"]
if "protocoltime" in df.columns:
    del df["protocoltime"]
df.head()

,instanceid,type,onoff,color,dimmer,weekday,weekofyear,year,month,day,time
0,65537,2,1,efd275,51,Friday,47,2020,11,20,22:00
1,65538,2,1,efd275,48,Friday,47,2020,11,20,22:00
2,65540,2,0,efd275,100,Friday,47,2020,11,20,22:00
3,65545,2,0,efd275,50,Friday,47,2020,11,20,22:00
4,65546,2,0,f1e0b5,50,Friday,47,2020,11,20,22:00


In [73]:
df.describe()

,instanceid,type,onoff,dimmer,weekofyear,year,month,day
count,1133.000000,1133.0,1133.000000,1133.000000,1133.0,1133.0,1133.0,1133.000000
mean,65548.847308,2.0,0.439541,75.323919,47.0,2020.0,11.0,20.960282
std,6.229460,0.0,0.496550,31.588064,0.0,0.0,0.0,0.195381
min,65537.000000,2.0,0.000000,0.000000,47.0,2020.0,11.0,20.000000
25%,65546.000000,2.0,0.000000,50.000000,47.0,2020.0,11.0,21.000000
50%,65549.000000,2.0,0.000000,100.000000,47.0,2020.0,11.0,21.000000
75%,65555.000000,2.0,1.000000,100.000000,47.0,2020.0,11.0,21.000000
max,65557.000000,2.0,1.000000,100.000000,47.0,2020.0,11.0,21.000000


In [74]:
df.isnull().sum()

instanceid    0
type          0
onoff         0
color         0
dimmer        0
weekday       0
weekofyear    0
year          0
month         0
day           0
time          0
dtype: int64

## Data preparation

In [75]:
from sklearn.model_selection import train_test_split

In [76]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=11)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=11)

In [77]:
y_train_onoff = (df_train.onoff == 1).values
y_val_onoff = (df_val.onoff == 1).values
y_test_onoff = (df_test.onoff == 1).values

In [78]:
del df_train["onoff"]
del df_val["onoff"]
del df_test["onoff"]

### One-Hot encoding

In [79]:
from sklearn.feature_extraction import DictVectorizer

In [80]:
dv = DictVectorizer(sparse=False)

In [81]:
dict_train_onoff = df_train.to_dict(orient="records")

In [85]:
X_train_onoff = dv.fit_transform(dict_train_onoff)

In [86]:
dv.vocabulary_

{'instanceid': 6,
 'type': 36,
 'color=f1e0b5': 2,
 'dimmer': 5,
 'weekday=Saturday': 38,
 'weekofyear': 39,
 'year': 40,
 'month': 7,
 'day': 4,
 'time=20:00': 30,
 'time=15:30': 19,
 'time=19:45': 29,
 'time=16:15': 22,
 'color=efd275': 1,
 'weekday=Friday': 37,
 'time=22:30': 34,
 'time=19:30': 28,
 'time=20:15': 31,
 'time=15:45': 20,
 'color=0': 0,
 'time=00:15': 9,
 'time=15:15': 18,
 'time=17:00': 24,
 'time=14:45': 16,
 'time=14:30': 15,
 'time=22:00': 33,
 'time=15:00': 17,
 'color=f5faf6': 3,
 'time=20:30': 32,
 'time=01:30': 10,
 'time=14:15': 14,
 'time=10:15': 11,
 'time=16:00': 21,
 'time=17:45': 25,
 'time=16:30': 23,
 'time=18:15': 26,
 'time=00:00': 8,
 'time=13:45': 13,
 'time=23:00': 35,
 'time=18:45': 27,
 'time=10:30': 12}